In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

# 1. Analyzing for Fictional Characters

### Checking amount of Fictional Characters

In [2]:
#  strings, which are then filled with queries, dependent on the users answers


sparql.setQuery("SELECT COUNT (DISTINCT ?x) WHERE { ?x a dbo:FictionalCharacter}")

try:
    ret = sparql.queryAndConvert()
    print("Number of Fictional Characters")
    for r in ret["results"]["bindings"]:
        print(r['callret-0']["value"])
except Exception as e:
    print(e)

Number of Fictional Characters
26000


### Subclasses existing in Fictional Character

In [3]:
sparql.setQuery("SELECT ?x,  COUNT( DISTINCT ?y) WHERE { ?x rdfs:subClassOf dbo:FictionalCharacter. ?y a dbo:FictionalCharacter. ?y a ?x }")

try:
    ret = sparql.queryAndConvert()
    print("Number of Fictional Characters")
    for r in ret["results"]["bindings"]:
        print( r['x']["value"], int(r['callret-1']["value"])  )
except Exception as e:
    print(e)

Number of Fictional Characters
http://dbpedia.org/ontology/ComicsCharacter 3559
http://dbpedia.org/ontology/SoapCharacter 6082
http://dbpedia.org/ontology/MythologicalFigure 1128


In [15]:
sparql.setQuery("SELECT COUNT(?x) WHERE { ?x a dbo:ComicsCharacter. ?x a dbo:FictionalCharacter} ")

try:
    ret = sparql.queryAndConvert()
    print("Number of Comic Characters")
    for r in ret["results"]["bindings"]:
        print( r['callret-0']["value"]  )
except Exception as e:
    print(e)

Number of Comic Characters
3559


In [14]:
sparql.setQuery("SELECT COUNT(?x) WHERE { ?x a dbo:MythologicalFigure. } ")

try:
    ret = sparql.queryAndConvert()
    print("Number of Mythological Figures (Less, when you add the type 'Fictional Character')")
    for r in ret["results"]["bindings"]:
        print(r['callret-0']["value"])
except Exception as e:
    print(e)

Number of Mythological Figures (Less, when you add the type 'Fictional Character'
2399


In [18]:
sparql.setQuery("SELECT COUNT(?x) WHERE { ?x a dbo:SoapCharacter. } ")

try:
    ret = sparql.queryAndConvert()
    print("Number of Soap Characters")
    for r in ret["results"]["bindings"]:
        print( r['callret-0']["value"]  )
except Exception as e:
    print(e)

Number of Soap Characters
6095


In [19]:
sparql.setQuery("SELECT COUNT(?x) WHERE { ?x a dbo:DisneyCharacter.} ")

try:
    ret = sparql.queryAndConvert()
    print("Number of Disney Characters")
    for r in ret["results"]["bindings"]:
        print( r['callret-0']["value"]  )
except Exception as e:
    print(e)

Number of Soap Characters
0


In [20]:
sparql.setQuery("SELECT COUNT(?x) WHERE { ?x a dbo:NarutoCharacter.} ")

try:
    ret = sparql.queryAndConvert()
    print("Number of Soap Characters")
    for r in ret["results"]["bindings"]:
        print( r['callret-0']["value"]  )
except Exception as e:
    print(e)

Number of Soap Characters
0


### Soap and Comic Characters should be the first questions to ask. If Soap Character is FALSE, then ask for Comic Characters. And so on. Maybe asking, whether the character is none of the mentioned subclasses could narrow down a lot of options. 

#### Analyzing Soap Character

Soap Character does not have any further subclasses. Algorithm to analyze the next properties those characters have. 


Now lets analyze the Comic Characters
There is only one subclass (2, but that one has arabic letters) 

In [10]:
sparql.setQuery("SELECT COUNT(?x) WHERE { ?x a dbo:AnimangaCharacter.} ")

try:
    ret = sparql.queryAndConvert()
    print("Number of Animanga Characters")
    for r in ret["results"]["bindings"]:
        print( r['callret-0']["value"]  )
except Exception as e:
    print(e)

Number of Animanga Characters
21


Well... That doesnt really help us... 

In [11]:
#  strings, which are then filled with queries, dependent on the users answers

select = "SELECT ?predicate ?object (COUNT(?x) as ?Appearances) "

#filter = "FILTER(regex(?season, '2021'))"
filter = ""

#where = "WHERE {?x ?predicate ?object. ?x a dbo:FormulaOneRacer. ?x dbp:lastSeason ?season. " + filter + "} "
where = "WHERE {?x ?predicate ?object. ?x a dbo:ComicsCharacter. } "

groupBy = "GROUP BY  ?predicate ?object "

orderBy = "ORDER BY DESC (COUNT(?x))"

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)






sparql.setQuery((select + where + groupBy + orderBy))
#sparql.setQuery("SELECT DISTINCT COUNT(?x) WHERE{?x rdf:type dbo:Athlete}")
 
resultSet = []

try:
        queryResult = sparql.queryAndConvert()

        for res in queryResult['results']['bindings']:
                #print(res)
                resultSet.append({"Object": res['object']['value'], "Predicate": res['predicate']
                          ['value'], "Appearances": int(res['Appearances']['value'])})
except Exception as e: 
        print(e)
    

for x in resultSet:
        print(x['Predicate'], "      ", x['Object'], "   ", x['Appearances'])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/ontology/ComicsCharacter     5164
http://dbpedia.org/property/wikiPageUsesTemplate        http://dbpedia.org/resource/Template:Reflist     3987
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://www.w3.org/2002/07/owl#Thing     3833
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#Agent     3629
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/ontology/Agent     3629
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://www.wikidata.org/entity/Q24229398     3629
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://www.wikidata.org/entity/Q95074     3559
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/ontology/FictionalCharacter     3559
http://dbpedia.org/property/wikiPageUsesTemplate        http://dbpedia.org/resource/Template:Infobox_comics_character     3112
http://www.w3.o

Analyzing this, Asking for the type of Imagintation or Cognition might be a bit too abstract for the user (Not sure we can avoid it. Might have to use it eventually (e.g. for Soap Characters)

The next best question is probably asking whether has a resource to "Marvel Comics" - As it is a harcoded question, we can ask normally, whether the chracter is a Marvel Character


In [15]:
#  strings, which are then filled with queries, dependent on the users answers

select = "SELECT ?predicate ?object (COUNT(?x) as ?Appearances) "

#filter = "FILTER(regex(?season, '2021'))"
filter = ""

#where = "WHERE {?x ?predicate ?object. ?x a dbo:FormulaOneRacer. ?x dbp:lastSeason ?season. " + filter + "} "
where = "WHERE {?x ?predicate ?object. ?x a dbo:ComicsCharacter. ?x <http://dbpedia.org/ontology/wikiPageWikiLink>  <http://dbpedia.org/resource/Marvel_Comics>. ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/class/yago/Ability105616246>. ?x <http://dbpedia.org/property/publisher> <http://dbpedia.org/resource/Marvel_Comics> } "

groupBy = "GROUP BY  ?predicate ?object "

orderBy = "ORDER BY DESC (COUNT(?x))"

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)






sparql.setQuery((select + where + groupBy + orderBy))
#sparql.setQuery("SELECT DISTINCT COUNT(?x) WHERE{?x rdf:type dbo:Athlete}")
 
resultSet = []

try:
        queryResult = sparql.queryAndConvert()

        for res in queryResult['results']['bindings']:
                #print(res)
                resultSet.append({"Object": res['object']['value'], "Predicate": res['predicate']
                          ['value'], "Appearances": int(res['Appearances']['value'])})
except Exception as e: 
        print(e)
    

for x in resultSet:
        print(x['Predicate'], "      ", x['Object'], "   ", x['Appearances'])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/ontology/ComicsCharacter     964
http://dbpedia.org/ontology/wikiPageWikiLink        http://dbpedia.org/resource/Marvel_Comics     964
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/class/yago/Imagination105625465     964
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/class/yago/FictionalCharacter109587565     964
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/class/yago/Ability105616246     964
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/class/yago/Creativity105624700     964
http://dbpedia.org/property/publisher        http://dbpedia.org/resource/Marvel_Comics     964
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/class/yago/Cognition100023271     964
http://www.w3.org/1999/02/22-rdf-syntax-ns#type        http://dbpedia.org/class/yago/PsychologicalFeature100023100     964
htt

# Analyzing Person

In [19]:
#  strings, which are then filled with queries, dependent on the users answers


sparql.setQuery("SELECT COUNT (DISTINCT ?x) WHERE { ?x a dbo:Person}")

try:
    ret = sparql.queryAndConvert()
    print("Number of Persons")
    for r in ret["results"]["bindings"]:
        print(r['callret-0']["value"])
except Exception as e:
    print(e)

Number of Persons
2230135


In [22]:
sparql.setQuery("SELECT ?x,  COUNT( DISTINCT ?y) WHERE { ?x rdfs:subClassOf dbo:Person. ?y a dbo:Person. ?y a ?x } ORDER BY DESC(COUNT(?y))")

try:
    ret = sparql.queryAndConvert()
    print("Number of Fictional Characters")
    for r in ret["results"]["bindings"]:
        print( r['x']["value"], int(r['callret-1']["value"])  )
except Exception as e:
    print(e)

Number of Fictional Characters
http://dbpedia.org/ontology/Athlete 538883
http://dbpedia.org/ontology/OrganisationMember 432612
http://dbpedia.org/ontology/Politician 189146
http://dbpedia.org/ontology/Artist 105168
http://dbpedia.org/ontology/OfficeHolder 56695
http://dbpedia.org/ontology/Writer 45114
http://dbpedia.org/ontology/Scientist 45023
http://dbpedia.org/ontology/MilitaryPerson 42880
http://dbpedia.org/ontology/SportsManager 27935
http://dbpedia.org/ontology/Cleric 23796
http://dbpedia.org/ontology/Royalty 20150
http://dbpedia.org/ontology/Coach 10802
http://dbpedia.org/ontology/Academic 9202
http://dbpedia.org/ontology/Noble 7544
http://dbpedia.org/ontology/Criminal 4981
http://dbpedia.org/ontology/Religious 4605
http://dbpedia.org/ontology/Architect 4567
http://dbpedia.org/ontology/Philosopher 2840
http://dbpedia.org/ontology/BeautyQueen 2680
http://dbpedia.org/ontology/Model 2007
http://dbpedia.org/ontology/Journalist 1843
http://dbpedia.org/ontology/Economist 1686
http://

### Pretty clear to search for Athlete, then Organization Member and then Politician. 

In [25]:
sparql.setQuery("SELECT ?x,  COUNT( DISTINCT ?y) WHERE { ?x rdfs:subClassOf dbo:Athlete. ?y a ?x} ORDER BY DESC(COUNT(?y))")

try:
    ret = sparql.queryAndConvert()
    print("Number of Fictional Characters")
    for r in ret["results"]["bindings"]:
        print( r['x']["value"], int(r['callret-1']["value"])  )
except Exception as e:
    print(e)

Number of Fictional Characters
http://dbpedia.org/ontology/SoccerPlayer 301369
http://dbpedia.org/ontology/Cricketer 37558
http://dbpedia.org/ontology/WinterSportPlayer 30276
http://dbpedia.org/ontology/BaseballPlayer 28728
http://dbpedia.org/ontology/GridironFootballPlayer 27775
http://dbpedia.org/ontology/RugbyPlayer 24788
http://dbpedia.org/ontology/BasketballPlayer 19373
http://dbpedia.org/ontology/Cyclist 16767
http://dbpedia.org/ontology/AustralianRulesFootballPlayer 13971
http://dbpedia.org/ontology/Swimmer 11706
http://dbpedia.org/ontology/TennisPlayer 9661
http://dbpedia.org/ontology/Boxer 8379
http://dbpedia.org/ontology/MotorsportRacer 8177
http://dbpedia.org/ontology/Wrestler 7821
http://dbpedia.org/ontology/MartialArtist 5074
http://dbpedia.org/ontology/VolleyballPlayer 5016
http://dbpedia.org/ontology/GaelicGamesPlayer 4833
http://dbpedia.org/ontology/HandballPlayer 4629
http://dbpedia.org/ontology/GolfPlayer 4526
http://dbpedia.org/ontology/Gymnast 4226
http://dbpedia.or

In case the Person is an Athlete, ask for a Soccer Athlete. This does not have any further subclasses

When looking at Organization Member, this has one subclass - SportsTeamMember. 

In [27]:
sparql.setQuery("SELECT ?x,  COUNT( DISTINCT ?y) WHERE { ?x rdfs:subClassOf dbo:OrganisationMember. ?y a ?x} ORDER BY DESC(COUNT(?y))")

try:
    ret = sparql.queryAndConvert()
    print("Number of Fictional Characters")
    for r in ret["results"]["bindings"]:
        print( r['x']["value"], int(r['callret-1']["value"])  )
except Exception as e:
    print(e)

Number of Fictional Characters
http://dbpedia.org/ontology/SportsTeamMember 432992


The question now. How many are both - Soccer Plaer and SportsTeam member? 

In [32]:
#  strings, which are then filled with queries, dependent on the users answers


sparql.setQuery("SELECT COUNT (?x) WHERE { ?x a dbo:SoccerPlayer. ?x a dbo:OrganisationMember}")

try:
    ret = sparql.queryAndConvert()
    print("Number of Persons")
    for r in ret["results"]["bindings"]:
        print(r['callret-0']["value"])
except Exception as e:
    print(e)

Number of Persons
0


None, what about Soccer Player and its subclass: SportsTeamMember

In [33]:

sparql.setQuery("SELECT COUNT (?x) WHERE { ?x a dbo:SoccerPlayer. ?x a dbo:SportsTeamMember}")

try:
    ret = sparql.queryAndConvert()
    print("Number of Persons")
    for r in ret["results"]["bindings"]:
        print(r['callret-0']["value"])
except Exception as e:
    print(e)

Number of Persons
0


Politician has way more subclasses again. So maybe its smarter to ask this questions, rather than for OrganizationMember

In [34]:
sparql.setQuery("SELECT ?x,  COUNT( DISTINCT ?y) WHERE { ?x rdfs:subClassOf dbo:Politician. ?y a ?x} ORDER BY DESC(COUNT(?y))")

try:
    ret = sparql.queryAndConvert()
    print("Number of Fictional Characters")
    for r in ret["results"]["bindings"]:
        print( r['x']["value"], int(r['callret-1']["value"])  )
except Exception as e:
    print(e)

Number of Fictional Characters
http://dbpedia.org/ontology/Congressman 1289
http://dbpedia.org/ontology/Mayor 1242
http://dbpedia.org/ontology/President 1030
http://dbpedia.org/ontology/Governor 871
http://dbpedia.org/ontology/MemberOfParliament 722
http://dbpedia.org/ontology/PrimeMinister 617
http://dbpedia.org/ontology/Ambassador 180
http://dbpedia.org/ontology/Senator 158
http://dbpedia.org/ontology/Chancellor 34


Looking at these numbers... They are not even close to adding up. So there are many people with type Politican, but not further Specification... 

As after here, we have to run our algorithm again. Maybe we specify special rules. Like: 

The first 60% of people we just analyze by type. Then after that we include other predicates. But lets talk about that!